# Önce modeli tanımlıyoruz;

### SORU 1:
  - Basit bir PyTorch modeli (SimpleNet) nasıl tanımlarım?
 - Bu modeli dummy bir input ile çalıştırarak forward pass'in sorunsuz olduğunu nasıl test ederim?

### ÇÖZÜM 1:
  - nn.Module'dan türeyen SimpleNet sınıfı tanımlıyorum.
  - Giriş boyutunu 10, çıkış boyutunu 3 sınıflı bir problem gibi ele alıyorum.
  - Dummy tensor ile forward yapıp çıktı shape'ini kontrol ediyorum.


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import os


class SimpleNet(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=32, num_classes=3):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: [batch_size, input_dim]
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


# Model örneği oluştur
model = SimpleNet(input_dim=10, hidden_dim=32, num_classes=3)

# Dummy input: batch_size = 1, input_dim = 10
dummy_input = torch.randn(1, 10)

# Forward test
with torch.no_grad():
    output = model(dummy_input)

print("Dummy input shape :", dummy_input.shape)
print("Output shape      :", output.shape)
print("Output tensor     :", output)


Dummy input shape : torch.Size([1, 10])
Output shape      : torch.Size([1, 3])
Output tensor     : tensor([[-0.1590,  0.2520, -0.2017]])


## Sonra modelin scriptini kayıt edelim :

In [4]:
model.eval()

scripted_model = torch.jit.script(model)
save_path = "model_ts.pt"
scripted_model.save(save_path)
print(f"TorchScript modeli kaydedildi: {os.path.abspath(save_path)}")

TorchScript modeli kaydedildi: c:\Users\hdgn5\OneDrive\Masaüstü\C ++\Bölüm - 11 ( LibTorch )\Uygulama - 1\model_ts.pt


----

## Main.cpp kodunun güncel halini bırakıyorum 

```cpp
#include <iostream>
#include <fstream>
#include <torch/script.h>

int main() {
    try {
        const std::string model_path = "model_ts.pt";

        std::cout << "[INFO] Program başladı.\n";
        std::cout << "[INFO] Beklenen model yolu: " << model_path << "\n";

        // 0) Dosya gerçekten var mı kontrol ediyorum
        {
            std::ifstream f(model_path);
            if (!f.good()) {
                std::cerr << "[HATA] model_ts.pt bulundu MU? -> HAYIR\n";
                std::cerr << "[HATA] model_ts.pt dosyasını exe ile aynı klasöre koymalısın.\n";
                std::cout << "Çıkmak için Enter'a bas...\n";
                std::cin.get();
                return -1;
            } else {
                std::cout << "[OK] model_ts.pt bulundu.\n";
            }
        }

        // 1) Modeli yükle
        torch::jit::script::Module module = torch::jit::load(model_path);
        std::cout << "[OK] Model yüklendi.\n";

        // 2) Cihaz (CPU)
        torch::Device device(torch::kCPU);
        module.to(device);
        module.eval();
        std::cout << "[OK] Model CPU'ya taşındı ve eval moduna alındı.\n";

        // 3) Dummy input
        torch::Tensor input = torch::randn({1, 10}, torch::TensorOptions().device(device));
        std::cout << "[OK] Dummy input olusturuldu. Shape: " << input.sizes() << "\n";

        // 4) Forward
        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(input);

        torch::Tensor output = module.forward(inputs).toTensor();

        std::cout << "[OK] Forward bitti.\n";
        std::cout << "Input shape : " << input.sizes() << std::endl;
        std::cout << "Output shape: " << output.sizes() << std::endl;
        std::cout << "Output tensor:\n" << output << std::endl;
    }
    catch (const c10::Error& e) {
        std::cerr << "[EXCEPTION] LibTorch hata fırlattı:\n" << e.what() << std::endl;
        std::cout << "Çıkmak için Enter'a bas...\n";
        std::cin.get();
        return -1;
    }

    std::cout << "Program bitti. Çıkmak için Enter'a bas...\n";
    std::cin.get();
    return 0;
}
```


# 🔥 Şimdi senden istediğim

Bu adımı uygulayalım:

1. OpenCV’yi CMakeLists’e ekle  
2. `main.cpp`’yi yukarıdaki gibi düzenle  
3. Bir `test.jpg` koy  
4. `cmake --build .`  
5. `model_test.exe` çalıştır

-----